<a href="https://colab.research.google.com/github/arthurvale/TCC-2026/blob/main/C%C3%B3pia_de_1_EDA_(filtro_de_campanha_e_PCIs_com_feixes_majorit%C3%A1rios).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

Mounted at /content/drive


# Filtragem do dataset.

In [ ]:
df = pd.read_parquet('/content/drive/MyDrive/5g final/dados/dataframe_cache.parquet')
display(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8144644 entries, 0 to 8144643
Data columns (total 37 columns):
 #   Column           Dtype  
---  ------           -----  
 0   Unnamed: 0       int64  
 1   Date             object 
 2   Time             object 
 3   UTC              float64
 4   Latitude         float64
 5   Longitude        float64
 6   Altitude         float64
 7   Speed            float64
 8   EARFCN           int64  
 9   Frequency        float64
 10  Band             int64  
 11  PCI              int64  
 12  SSBIdx           int64  
 13  SSS-SINR         float64
 14  SSS-RSRP         float64
 15  SSS-RSRQ         float64
 16  SSS-RePower      float64
 17  MNC              object 
 18  DM_RS-SINR       float64
 19  DM_RS-RSRP       float64
 20  DM_RS-RSRQ       float64
 21  DM_RS-RePower    float64
 22  PBCH-SINR        float64
 23  PBCH-RSRP        float64
 24  PBCH-RSRQ        float64
 25  PBCH-RePower     float64
 26  PSS-SINR         float64
 27  PSS-RSRP    

None

Descrição das Variáveis Categóricas (Artigo Referência):

- 'MNC': Identificador da operadora de rede móvel
- 'scenario': Contexto da coleta de dados:
  * 'OW' (Outdoor Walking): Coleta realizada a pé em ambiente externo.
  * 'OD' (Outdoor Driving): Coleta em veículo em movimento (cenário dinâmico).
  * 'IS' (Indoor Static): Coleta estática em ambiente interno.
- 'campaign': Subcampanhas de medição realizadas em diferentes dias e condições.
  Cada subcampanha é identificada por um código único (ex: 'OD_1', 'OD_gaming').



In [ ]:
unicos_mnc = df['MNC'].unique()
print(unicos_mnc)
print('------------------------------------------------------------------------')
unicos_scenario = df['scenario'].unique()
print(unicos_scenario)
print('------------------------------------------------------------------------')
unicos_campaign = df['campaign'].unique()
print(unicos_campaign)

['"Op"[2]' '"Op"[1]']
------------------------------------------------------------------------
['OW' 'IS' 'OD']
------------------------------------------------------------------------
['campaign_6_OW_5G_gaming' 'campaign_35_IS_5G_speedtest'
 'campaign_10_OD_5G_speedtest' 'campaign_28_OW_5G_gaming'
 'campaign_20_OW_5G_gaming' 'campaign_18_OD_5G_gaming'
 'campaign_7_OD_5G_gaming' 'campaign_5_OW_5G_gaming'
 'campaign_29_OD_5G_gaming' 'campaign_4_OD_5G_gaming'
 'campaign_23_OD_5G_gaming' 'campaign_10_IS_5G_gaming'
 'campaign_27_IS_5G_gaming' 'campaign_9_IS_5G_gaming'
 'campaign_8_IS_5G_gaming' 'campaign_13_OW_5G_gaming'
 'campaign_24_OW_5G_gaming' 'campaign_31_OW_5G_gaming'
 'campaign_21_IS_5G_gaming' 'campaign_22_IS_5G_gaming'
 'campaign_33_IS_5G_gaming' 'campaign_12_OD_5G_gaming'
 'campaign_26_OD_5G_gaming' 'campaign_2_OD_5G_gaming'
 'campaign_14_IS_5G_gaming' 'campaign_15_IS_5G_gaming'
 'campaign_1_IS_5G_gaming' 'campaign_11_IS_5G_gaming'
 'campaign_34_IS_5G_gaming' 'campaign_32_IS_5G_

Nosso estudo de caso é sobre a classificação de Beamforming, neste contexto o tipo de campanha que oferece mais diversidade de condições de rede é a campanha com o veículo em movimento, OD, e sem testes ativos, como gammimg e speedtest.

In [ ]:
df_ow=df[df['scenario']=='OD']
df_ow_5g = df_ow[~df_ow['campaign'].str.endswith(('_gaming', '_speedtest'))]

In [ ]:
df_ow_5g.shape

(604630, 37)

Verificação dos feixes nas redes das operadoras. Todas as operadoras estão usando multi-beams ou usam single-beam ?

In [ ]:
unicos_mnc = df_ow_5g['MNC'].unique()
target_op2 = unicos_mnc[0]
target_op1 = unicos_mnc[1]
df_filtered_op2 = df_ow_5g[df_ow_5g['MNC'] == target_op2]

df_filtered_op1 = df_ow_5g[df_ow_5g['MNC'] == target_op1]
print('------------------------------------------------------------------------')
print(f'Operadora: {target_op1}')
ssb_counts_op1 = df_filtered_op1['SSBIdx'].value_counts().reset_index()
display(ssb_counts_op1)
ssb_counts_op2 = df_filtered_op2['SSBIdx'].value_counts().reset_index()
print('------------------------------------------------------------------------')
print(f'Operadora: {target_op2}')
display(ssb_counts_op2)

------------------------------------------------------------------------
Operadora: "Op"[1]


,SSBIdx,count
0,0,74386


------------------------------------------------------------------------
Operadora: "Op"[2]


,SSBIdx,count
0,2,74176
1,3,70235
2,1,69638
3,0,68062
4,5,64252
5,4,61864
6,7,61270
7,6,60747


Portanto, a rede da operadora ' "Op"[1] ' possui apenas 1 feixe e a rede da operadora ' "Op"[2] ' possui mais feixes, portanto apenas a segunda operadora tem utilidade para o ontext ode classificação de feixes.

In [ ]:
df_ow_5g_op2=df_ow_5g[df_ow_5g['MNC']==target_op2]

In [ ]:
df_ow_5g_op2.shape

(530244, 37)

In [ ]:
file_path = '/content/drive/MyDrive/5g final/dados/df_parte1.parquet'
df_ow_5g_op2.to_parquet(file_path)
print(f"DataFrame salvo em: {file_path}")

DataFrame salvo em: /content/drive/MyDrive/5g final/dados/df_parte1.parquet


In [ ]:
df_ow_5g_op2.columns

Index(['Unnamed: 0', 'Date', 'Time', 'UTC', 'Latitude', 'Longitude',
       'Altitude', 'Speed', 'EARFCN', 'Frequency', 'Band', 'PCI', 'SSBIdx',
       'SSS-SINR', 'SSS-RSRP', 'SSS-RSRQ', 'SSS-RePower', 'MNC', 'DM_RS-SINR',
       'DM_RS-RSRP', 'DM_RS-RSRQ', 'DM_RS-RePower', 'PBCH-SINR', 'PBCH-RSRP',
       'PBCH-RSRQ', 'PBCH-RePower', 'PSS-SINR', 'PSS-RSRP', 'PSS-RSRQ',
       'PSS-RePower', 'SS_PBCH-SINR', 'SS_PBCH-RSRP', 'SS_PBCH-RSRQ',
       'SS_PBCH-RePower', 'scenario', 'distance_w', 'campaign'],
      dtype='object')

# Filtro do Scanner a cada instante

Transformando a coluna de datas

In [ ]:
df_ow_5g_op2['Time'] = pd.to_timedelta(df_ow_5g_op2['Time'])
df_ow_5g_op2['DateTime'] = pd.to_datetime(df_ow_5g_op2['Date'], format='%d.%m.%Y') + df_ow_5g_op2['Time']
df_ow_5g_op2 = df_ow_5g_op2.sort_values('DateTime')


/tmp/ipython-input-7-2486577727.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ow_5g_op2['Time'] = pd.to_timedelta(df_ow_5g_op2['Time'])
/tmp/ipython-input-7-2486577727.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ow_5g_op2['DateTime'] = pd.to_datetime(df_ow_5g_op2['Date'], format='%d.%m.%Y') + df_ow_5g_op2['Time']


Verifica-se que para cada isntante de medição há mais de um sinal registrado. Isto ocorre pois o instrumento de emdição é um scanner de rede, ele não é um aparelho que inidica qual a conexao atual, apenas verifica quais as possibilidades em determinado ponto da medição.


In [ ]:
min_datetime = df_ow_5g_op2['DateTime'].min()
df_ow_5g_op2[df_ow_5g_op2['DateTime'] == min_datetime]

,Unnamed: 0,Date,Time,UTC,Latitude,Longitude,Altitude,Speed,EARFCN,Frequency,...,PSS-RSRQ,PSS-RePower,SS_PBCH-SINR,SS_PBCH-RSRP,SS_PBCH-RSRQ,SS_PBCH-RePower,scenario,distance_w,campaign,DateTime
4747450,4310089,14.12.2020,0 days 18:49:24.612000,1.610647e+09,41.89322,12.49277,51.44,5.33,643296,3649.44,...,-18.4,-120.0,-7.7,-128.0,-19.1,-119.6,OD,0.0,campaign_34_OD_5G,2020-12-14 18:49:24.612
4747440,3310390,14.12.2020,0 days 18:49:24.612000,1.610647e+09,41.89322,12.49277,51.44,5.33,643296,3649.44,...,-14.3,-121.7,-1.7,-125.3,-14.9,-121.3,OD,0.0,campaign_34_OD_5G,2020-12-14 18:49:24.612
4747438,3110291,14.12.2020,0 days 18:49:24.612000,1.610647e+09,41.89322,12.49277,51.44,5.33,643296,3649.44,...,-20.8,-120.0,-6.8,-127.3,-18.3,-119.6,OD,0.0,campaign_34_OD_5G,2020-12-14 18:49:24.612
4747427,2010900,14.12.2020,0 days 18:49:24.612000,1.610647e+09,41.89322,12.49277,51.44,5.33,643296,3649.44,...,-24.2,-122.2,-16.8,-138.2,-27.6,-121.4,OD,0.0,campaign_34_OD_5G,2020-12-14 18:49:24.612


Portanto, deve-se filtrar cada um destes instantes, o critério usado sera a maior potencia do sinal, 'SS_PBCH-RSRP', e se houver algum momento em que a pontencia dos sinais for igual, o sinal escolhido será o de maior 'SS_PBCH-RSRQ'.

In [ ]:
def select_best_rsrp_rsrq(group):
    max_rsrp = group['SS_PBCH-RSRP'].max()

    max_rsrp_rows = group[group['SS_PBCH-RSRP'] == max_rsrp]

    if len(max_rsrp_rows) == 1:
        return max_rsrp_rows

    max_rsrq = max_rsrp_rows['SS_PBCH-RSRQ'].max()
    best_row = max_rsrp_rows[max_rsrp_rows['SS_PBCH-RSRQ'] == max_rsrq].head(1)

    return best_row


df_ow_5g_op2_unique_datetime = (df_ow_5g_op2.groupby('DateTime', group_keys=False).apply(select_best_rsrp_rsrq).reset_index(drop=True))


/tmp/ipython-input-9-3228435935.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_ow_5g_op2_unique_datetime = (df_ow_5g_op2.groupby('DateTime', group_keys=False).apply(select_best_rsrp_rsrq).reset_index(drop=True))


In [ ]:
print("Verificando valores nulos:")
print(df_ow_5g_op2_unique_datetime.isnull().sum())

print("\nVerificando registros duplicados:")
print(df_ow_5g_op2_unique_datetime.duplicated().sum())

Verificando valores nulos:
Unnamed: 0         0
Date               0
Time               0
UTC                0
Latitude           0
Longitude          0
Altitude           0
Speed              0
EARFCN             0
Frequency          0
Band               0
PCI                0
SSBIdx             0
SSS-SINR           0
SSS-RSRP           0
SSS-RSRQ           0
SSS-RePower        0
MNC                0
DM_RS-SINR         0
DM_RS-RSRP         0
DM_RS-RSRQ         0
DM_RS-RePower      0
PBCH-SINR          0
PBCH-RSRP          0
PBCH-RSRQ          0
PBCH-RePower       0
PSS-SINR           0
PSS-RSRP           0
PSS-RSRQ           0
PSS-RePower        0
SS_PBCH-SINR       0
SS_PBCH-RSRP       0
SS_PBCH-RSRQ       0
SS_PBCH-RePower    0
scenario           0
distance_w         0
campaign           0
DateTime           0
dtype: int64

Verificando registros duplicados:
0
